#Installing neccessary libraries

In [ ]:
%pip -q install google-cloud-bigquery
%pip install pandas-gbq -U
%pip install catboost
%pip install pmdarima
%pip install prophet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing neccessary libraries 

In [ ]:
import os 
from google.cloud import bigquery 
import pandas as pd 
import pandas_gbq 
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split 
import plotly.graph_objs as go 
from prophet import Prophet 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 


# Setting up BigQuery client


In [ ]:
key_file = 'oreilly_demo_key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_file
client = bigquery.Client()

In [ ]:
import json 

f = open(key_file)
data = json.load(f)
project_id = data['project_id']
f.close()


#Querying the google analytics table 

In [ ]:
query1 = """
SELECT 
date, totals.visits, 
hits.hitNumber, totals.transactionRevenue, 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
UNNEST(hits) as hits 
WHERE totals.transactionRevenue >= 10
LIMIT 10 
"""

df = pandas_gbq.read_gbq(query1, project_id = project_id)
df.head()


Downloading: 100%|██████████|


,date,visits,hitNumber,transactionRevenue
0,20170402,1,1,8990000
1,20170402,1,2,8990000
2,20170402,1,3,8990000
3,20170402,1,4,8990000
4,20170402,1,5,8990000


In [ ]:
query1 = """
SELECT 
date, 
totals.transactionRevenue, 
totals.pageviews, 
totals.visits, 
device.browser,
geonetwork.country,
trafficSource.source, 
hits.hitNumber, 
fullVisitorId,
visitId,
visitNumber,

FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
UNNEST(hits) as hits
WHERE 
totals.transactionRevenue >= 0 AND 
_TABLE_SUFFIX BETWEEN '20170101' AND '20170731'

LIMIT 10
        """

df = pandas_gbq.read_gbq(query1, project_id=project_id)
df.head()

Downloading: 100%|██████████|


,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber
0,20170130,87990000,14,1,Chrome,United States,(direct),1,4307411610497308919,1485812030,3
1,20170130,87990000,14,1,Chrome,United States,(direct),2,4307411610497308919,1485812030,3
2,20170130,87990000,14,1,Chrome,United States,(direct),3,4307411610497308919,1485812030,3
3,20170130,87990000,14,1,Chrome,United States,(direct),4,4307411610497308919,1485812030,3
4,20170130,87990000,14,1,Chrome,United States,(direct),5,4307411610497308919,1485812030,3


In [ ]:
print(df.columns)

Index(['visitorId', 'visitNumber', 'visitId', 'visitStartTime', 'date',
       'totals', 'trafficSource', 'device', 'geoNetwork', 'customDimensions',
       'hits', 'fullVisitorId', 'userId', 'clientId', 'channelGrouping',
       'socialEngagementType'],
      dtype='object')


In [ ]:
df.to_csv("sample_ga_data500k_test.csv", index=False)

In [27]:
df_read_ga = pd.read_csv("sample_ga_data500K.csv")
df_read_ga.head()

,date,transactionRevenue,pageviews,visits,browser,country,source,hitNumber,fullVisitorId,visitId,visitNumber
0,20170216,33590000,8,1,Chrome,United States,(direct),1,3519741939483828661,1487262855,3
1,20170216,33590000,8,1,Chrome,United States,(direct),2,3519741939483828661,1487262855,3
2,20170216,33590000,8,1,Chrome,United States,(direct),3,3519741939483828661,1487262855,3
3,20170216,33590000,8,1,Chrome,United States,(direct),4,3519741939483828661,1487262855,3
4,20170216,33590000,8,1,Chrome,United States,(direct),5,3519741939483828661,1487262855,3


In [28]:
df_read_ga['date'] = pd.to_datetime(df_read_ga['date'], format="%Y%m%d") #'20170216'
df_ga_views  = df_read_ga.groupby(['date'])['pageviews'].sum().reset_index()

#Line plot of page views over time 

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_ga_views['date'], y= df_ga_views['pageviews'], mode='lines', name = "Time Series"))
fig.update_layout(title = "Page Views over Time", xaxis_title='Date', yaxis_title="Page Views")
fig.show()

# Line plot of visits 

In [31]:
df_ga_visits = df_read_ga.groupby(['date'])['visits'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ga_visits['date'], y=df_ga_visits['visits'], mode='lines', name='Time Series'))
fig.update_layout(title="Visits over time", xaxis_title='Date', yaxis_title='visits')
fig.show()

# Histograms of Page Views & Visits

In [32]:
fig = px.histogram(df_ga_visits, x="visits")
fig.show()

In [33]:
fig = px.histogram(df_ga_views, x="pageviews")
fig.show()

# Pie chart of visits by country 

In [36]:
df_visits_country = df_read_ga.groupby(['country'])['visits'].sum().reset_index()
df_visits_country.loc[df_visits_country['visits'] <500 , 'country' ] = 'Other Countries'
fig = px.pie(df_visits_country, values='visits', names='country', title='Visits by Country')
fig.show()